## <span style="color:grey">Initialization</span>

In [1]:
%run "KiteConnect_AlgoBase.ipynb" 

##  <span style="color:green">Trading Strategy </span>

In [64]:
class algoTrade(algoTrade_base):
   
    def __init__(s, symbol):
        logger.debug("AlgoTrade Called: "+symbol)
        super(algoTrade, s).__init__(symbol)
        
    
    # Long Strategies
    def long_indicators(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
        temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
        temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
        temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
        temp_df = temp_df | (REF(s.rsi,1) <=50)
        #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
        #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)
        
        return pd.DataFrame(  temp_df, columns=["buy"] )
     
    # Short Strategies
    def short_indicators(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
        temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
        temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
        temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
        temp_df = temp_df | (REF(s.rsi,1) >=50)
        #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
        return pd.DataFrame( temp_df, columns=["sell"])
    

    
    def long_breakout(s):
        temp_df = s.haCLOSE >= s.BBT.shift(1)
        temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
        temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_breakout(s):
        temp_df = s.haOPEN <= s.BBB.shift(1)
        temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
        temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
        return pd.DataFrame( temp_df , columns=["sell"])
    
    def long_ha(s):
        temp_df = (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN) 
        #temp_df = temp_df & (s.rsi < 40 )
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_ha(s):
        temp_df = (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
        #temp_df = temp_df & (s.rsi > 60 )
        return pd.DataFrame( temp_df , columns=["sell"])

    def tradeDecision(s, price_ohlc_df):
        global blackoutEnabled

        super(algoTrade, s).tradeDecision(price_ohlc_df)
        
        #s.rsi = RSI(s.CLOSE, timeperiod=20)
        #s.min = MIN(s.CLOSE, timeperiod=30)
        #s.max = MAX(s.CLOSE, timeperiod=30)
        
        s.tenkan  = (MIN(s.LOW, timeperiod=9)+MAX(s.HIGH, timeperiod=9))/2
        s.kijun   = (MIN(s.LOW, timeperiod=26)+MAX(s.HIGH, timeperiod=26))/2
        s.senokuA = REF((s.tenkan+s.kijun)/2,26)
        s.senokuB = REF((MIN(s.LOW, timeperiod=52)+MAX(s.HIGH, timeperiod=52))/2,26)
        s.chikou = REF(s.CLOSE, -26)
        
        s.buy  = s.long_ha() & s.long_indicators()
        s.sell = s.short_ha() & s.short_indicators()
        
        s.postprocess()
        
        return (s.buy,s.sell)
    
            
    def plot(s, symbol, noofcandles=120, ha=False):
        init_notebook_mode(connected=True)
        fig = tools.make_subplots(rows=1, cols=1, shared_xaxes=True, row_width=[1], vertical_spacing = 0.05)
        fig['layout']['xaxis'] = dict(rangeslider = dict(visible=False), side="bottom") #, range=[xMin,xMax])
        fig['layout'].update(height=750, plot_bgcolor='rgba(0,0,0,0)', title="Charts for "+symbol)
        
        fig['layout']['yaxis']['anchor'] = 'x'
        fig['layout']['yaxis']['side'] = 'right'

        fig['layout']['xaxis']['rangeselector'] = dict(
                    buttons=list([dict(count=1, label='1h', step='hour', stepmode='backward'),
                                  dict(count=2, label='2h', step='hour', stepmode='backward'),
                                  dict(count=3, label='3h', step='hour', stepmode='backward'),
                                  dict(count=6, label='4h', step='hour', stepmode='backward'),
                                  dict(count=6, label='1d', step='hour', stepmode='backward'),
                                  dict(step='all')]))
        
        yMin = s.LOW.iloc[-1*noofcandles:-1].min()*0.99
        yMax = s.HIGH.iloc[-1*noofcandles:-1].max()*1.001

        xMin = s.CLOSE.index[-1*noofcandles]
        xMax = s.CLOSE.index[-1]
        fig['layout']['yaxis']['range'] = [yMin, yMax]
        fig['layout']['xaxis']['range'] = [xMin, xMax]
        
        #print(s.buy.shape)
        #print(s.sell.shape)
        
        
        #print("plot")
        
        traceBuy=go.Scatter(x=s.buy.index.astype('str'), y=-1 * s.buy.close, name='BUY', mode="markers",marker = dict(color = 'rgba(119, 221, 119, 0.2)', size = 25, symbol='circle', line = dict(color = 'rgb(119, 221, 119)', width = 3)),showlegend=False)
        
        traceSell=go.Scatter(x=s.sell.index.astype('str'), y=s.sell.close, name='SELL', mode="markers",marker = dict(color = 'rgba(255, 0, 0,0.2)',size = 25, symbol='circle', line = dict(color = 'rgb(255, 0, 0)', width = 3)),showlegend=False)
       
        if ha:
            trace = go.Candlestick(x=s.CLOSE.index.astype('str'), open=s.haOPEN, high=s.haHIGH, low=s.haLOW, close=s.haCLOSE, name="Heikin Ashi", showlegend=False, hoverinfo = 'x+y+text', hovertext=s.hovertextarray)        
        else:
            trace = go.Candlestick(x=s.CLOSE.index.astype('str'), open=s.OPEN, high=s.HIGH, low=s.LOW, close=s.CLOSE, name="Candlestick", showlegend=False, hoverinfo = 'x+y+text', hovertext=s.hovertextarray)      
        
        
        traceTenkan = go.Scatter(x=s.tenkan.index.astype('str'), y=s.tenkan, name='Tenkan',  line=dict(color='cyan'),showlegend=False, hoverinfo = 'none')
        traceKijun = go.Scatter(x=s.kijun.index.astype('str'), y=s.kijun, name='Kijun',  line=dict(color='fuchsia'),showlegend=False, hoverinfo = 'none')
        traceChikou = go.Scatter(x=s.chikou.index.astype('str'), y=s.chikou, name='Chikou',  line=dict(color='orange'),showlegend=False, hoverinfo = 'none')
        
        
        s.temp1 = s.senokuA[s.senokuA>s.senokuB]
        s.temp2 = s.senokuB[s.senokuA<=s.senokuB]
        
        s.temp1 = s.senokuA[s.senokuA>s.senokuB].append(s.senokuB[s.senokuA<=s.senokuB]).sort_index()
        s.temp2 = s.senokuA[s.senokuA<s.senokuB].append(s.senokuB[s.senokuA>=s.senokuB]).sort_index()
        #s.temp1.sort_index()
        
        traceSenokuA = go.Scatter(x=s.temp1.index.astype('str'), y=s.temp1, name='Senoku A',  line=dict(color='rgba(0,0,0,0)'),showlegend=False, hoverinfo = 'none', fill = 'tonexty', fillcolor="rgba(18,230,3,0.4)")
        traceSenokuA2 = go.Scatter(x=s.temp2.index.astype('str'), y=s.temp2, name='Senoku A',  line=dict(color='rgba(0,0,0,0)'),showlegend=False, hoverinfo = 'none')
        traceSenokuB = go.Scatter(x=s.senokuB.index.astype('str'), y=s.senokuB, name='Senoku B',  line=dict(color='rgba(0,0,0,0)'),showlegend=False, hoverinfo = 'none', fill = 'tonexty', fillcolor="rgba(230,18,3,0.4)")
        
        #traceMACD = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.macd, name='MACD', line=dict(color='black'),showlegend=False)
        #traceMACDSignal = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.macdsignal, name='MACD signal', line=dict(color='red'),showlegend=False)
        #traceMACDHist = go.Bar(x=s.CLOSE.index.astype('str'), y=s.macdhist, name='MACD Hist', marker=dict(color="grey"),showlegend=False)
        #traceSK  = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.fastk, name='%K', line=dict(color='black'), yaxis='y3',showlegend=False)
        #traceSD  = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.fastd, name='%D', line=dict(color='red'),showlegend=False)
        #traceBBT = go.Scatter(x=s.BBT.index.astype('str'), y=s.BBT, name='BB_up',  line=dict(color='lightgrey'),showlegend=False, hoverinfo = 'none')
        #traceBBB = go.Scatter(x=s.BBB.index.astype('str'), y=s.BBB, name='BB_low',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False, hoverinfo = 'none')
        #traceBBM = go.Scatter(x=s.BBM.index.astype('str'), y=s.BBM, name='BB_mid',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False, hoverinfo = 'none')
        
        #traceMIN = go.Scatter(x=s.min.index.astype('str'), y=s.min, name='MIN',  line=dict(color='darkblue'), showlegend=False, hoverinfo = 'none')
        #traceMAX = go.Scatter(x=s.max.index.astype('str'), y=s.max, name='MAX',  line=dict(color='darkcyan'), showlegend=False, hoverinfo = 'none')
        
        #traceRSI = go.Scatter(x=s.rsi.index.astype('str'), y=s.rsi, name='RSI',  line=dict(color='darkcyan'), showlegend=False)
        
        
        #fig.append_trace(traceBBT, 1, 1)
        #fig.append_trace(traceBBB, 1, 1)
        #fig.append_trace(traceBBM, 1, 1)
        
        
        fig.append_trace(traceSenokuA2, 1, 1)
        fig.append_trace(traceSenokuB, 1, 1)
        fig.append_trace(traceSenokuA, 1, 1)
        
        fig.append_trace(traceTenkan, 1, 1)
        fig.append_trace(traceKijun, 1, 1)
        fig.append_trace(traceChikou, 1, 1)
        
        
        
        #fig.append_trace(traceBuy, 1, 1)
        #fig.append_trace(traceSell, 1, 1)
        
        #fig.append_trace(traceMIN, 1, 1)
        #fig.append_trace(traceMAX, 1, 1)
    
        #fig.append_trace(traceSK, 2, 1)
        #fig.append_trace(traceSD, 2, 1)
        
        
        
        #fig['layout']['yaxis2']['title']="Stochastics"
        #fig['layout']['yaxis2']['side']="right"
        #fig['layout']['yaxis2']['anchor']="x"
        #fig['layout']['yaxis2']['tickvals']=[20,80,30,70]
        
        #fig.append_trace(traceMACD, 3, 1)
        #fig.append_trace(traceMACDSignal, 3, 1)
        #fig.append_trace(traceMACDHist, 3, 1)
        fig.append_trace(trace, 1, 1)
        
        #fig['layout']['yaxis3']['anchor']="x"
        #fig['layout']['yaxis3']['side']="right"
        #fig['layout']['yaxis3']['title']="MACD"
        fig['layout']['yaxis1']['title']="Candlestick"
        
        
        #fig.append_trace(traceRSI, 4, 1)
        #fig['layout']['yaxis4']['title']="RSI"
        #fig['layout']['yaxis4']['side']="right"
        #fig['layout']['yaxis4']['anchor']="x"
        #fig['layout']['yaxis4']['tickvals']=[30,50,70]
        
        iplot(fig, filename="plot/"+x+".html")
        
if mode == "Algo":
    x = 'YESBANK'
    gc.collect()
    days=1
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,5,3,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate, cd=False, plot=True, ha=False)+"%")
    
    

   YESBANK 	 3.85 	 2.19%


This is the format of your plot grid:
[ (1,1) x1,y1 ]




Portfolio return: 2.19%


In [ ]:
# Backtesting for no of days

if mode == "Algo":
    x = 'YESBANK'
    gc.collect()
    days=2
    portfolio = pd.DataFrame([x])
    
    noofdays = 180
    for i in np.linspace(noofdays,1,noofdays):
        toDate = dt.datetime(2019,5,2,15,15) - dt.timedelta(days=i)
        #print(toDate)
        if isholiday(toDate) == True:
            continue
        print(toDate)
        #portfolio = pd.DataFrame(downloadlist)    
        trade_simulator(portfolio, toDate, cd=False, plot=False)
    
    

In [ ]:
if mode == "Algo":
    x = 'RELIANCE'
    gc.collect()
    days =2
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,4,30,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate, cd=True, plot=True)+"%")

In [ ]:
if mode == "Algo":
    x = 'INFRATEL'
    gc.collect()
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,4,30,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate, mode="simulator")+"%")